## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
1,8,Hilo,US,75.79,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
2,11,Sabang,ID,80.83,overcast clouds,5.8933,95.3214,Hotel Citra
3,24,Rikitea,PF,79.09,overcast clouds,-23.1203,-134.9692,People ThankYou
4,29,Cabo San Lucas,MX,75.51,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0,30.1),zoom_level = 1.7)
heatlayer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,max_intensity=200, point_radius=4)
fig.add_layer(heatlayer)

In [4]:
# 4a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["Country"] == "AU") & (vacation_df["Max Temp"] >= 78) & (vacation_df["Max Temp"] <= 80)]
vacation_end = vacation_df.loc[(vacation_df["Country"] == "AU") & (vacation_df["Max Temp"] >= 78) & (vacation_df["Max Temp"] <= 80)]
vacation_stop1 = vacation_df.loc[(vacation_df["Country"] == "AU") & (vacation_df["Max Temp"] >= 80) & (vacation_df["Max Temp"] <= 80.1)]
vacation_stop2 = vacation_df.loc[(vacation_df["Country"] == "AU") & (vacation_df["Max Temp"] >= 80.5) & (vacation_df["Max Temp"] <= 83)]
vacation_stop3 = vacation_df.loc[(vacation_df["Country"] == "AU") & (vacation_df["Max Temp"] >= 83)]

vacation_df.loc[(vacation_df["Country"] == "AU")]

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
44,160,Carnarvon,AU,77.07,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
51,186,Maningrida,AU,77.83,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge
54,190,Alyangula,AU,80.35,overcast clouds,-13.8483,136.4192,Groote Eylandt Lodge
108,430,Mount Isa,AU,78.57,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
111,452,Port Hedland,AU,82.45,overcast clouds,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
132,534,Gladstone,AU,76.93,clear sky,-23.8500,151.2500,Highpoint International Hotel
143,575,Cairns,AU,80.06,scattered clouds,-16.9167,145.7667,Mantra Trilogy Cairns
144,576,Karratha,AU,83.61,broken clouds,-20.7377,116.8463,ibis Styles Karratha


In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = vacation_start.to_numpy()
end = vacation_end.to_numpy()
stop1 = vacation_stop1.to_numpy()
stop2 = vacation_stop2.to_numpy()
stop3 = vacation_stop3.to_numpy()


lats = [start[0][5],end[0][5],stop1[0][5],stop2[0][5],stop3[0][5]]
lngs = [start[0][6],end[0][6],stop1[0][6],stop2[0][6],stop3[0][6]]

itinery = list(zip(lats,lngs))

itinery

[(-20.7333, 139.5),
 (-20.7333, 139.5),
 (-16.9167, 145.7667),
 (-20.3167, 118.5667),
 (-20.7377, 116.8463)]

In [30]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
start = itinery[0]
end = itinery[1]
stop1 = itinery[2]
stop2 = itinery[3]
stop3 = itinery[4]

fig = gmaps.figure()
itinery_route = gmaps.directions_layer(
    start,end,waypoints = [stop1,stop2,stop3],
    travel_mode = "DRIVING"
)
fig.add_layer(itinery_route)
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,430,Mount Isa,AU,78.57,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
1,575,Cairns,AU,80.06,scattered clouds,-16.9167,145.7667,Mantra Trilogy Cairns
2,452,Port Hedland,AU,82.45,overcast clouds,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
3,576,Karratha,AU,83.61,broken clouds,-20.7377,116.8463,ibis Styles Karratha


In [45]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat","Lng"]]

# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig = gmaps.figure()
itinery_route = gmaps.directions_layer(
    start,end,waypoints = [stop1,stop2,stop3],
    travel_mode = "DRIVING", show_markers=False
)
fig.add_layer(marker_layer)
fig.add_layer(itinery_route)

fig

Figure(layout=FigureLayout(height='420px'))